## Q.31. アフィン変換(スキュー)

(1)アフィン変換を用いて、出力(1)のようなX-sharing(dx = 30)画像を作成せよ。

(2)アフィン変換を用いて、出力2のようなY-sharing(dy = 30)画像を作成せよ。

(3)アフィン変換を用いて、出力3のような幾何変換した(dx = 30, dy = 30)画像を作成せよ。

このような画像はスキュー画像と呼ばれ、画像を斜め方向に伸ばした画像である。

出力(1)の場合、x方向にdxだけ引き伸ばした画像はX-sharingと呼ばれる。

出力(2)の場合、y方向にdyだけ引き伸ばした画像はY-sharingと呼ばれる。

それぞれ次式のアフィン変換で実現できる。
ただし、元画像のサイズがh x wとする。

```bash
(1) X-sharing                  (2) Y-sharing
   a = dx / h                     a = dy / w

  x'       1 a tx    x           x'       1 0 tx    x
[ y' ] = [ 0 1 ty ][ y ]       [ y' ] = [ a 1 ty ][ y ]
  1        0 0  1    1           1        0 0  1    1
```

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Affine
def affine(img, dx=30, dy=30):
  H, W, C = img.shape

  # Affine hyper parameters
  a = 1.
  b = dx / H
  c = dy / W
  d = 1.
  tx = 0.
  ty = 0.

  # prepare temporary
  _img = np.zeros((H+2, W+2, C), dtype=np.float32)

  # insert image to center of temporary
  _img[1:H+1, 1:W+1] = img

  # prepare affine image temporary
  H_new = np.ceil(dy + H).astype(np.int16)
  W_new = np.ceil(dx + W).astype(np.int16)
  out = np.zeros((H_new, W_new, C), dtype=np.float32)

  # preprare assigned index
  x_new = np.tile(np.arange(W_new), (H_new, 1))
  y_new = np.arange(H_new).repeat(W_new).reshape(H_new, -1)

  # prepare inverse matrix for affine
  adbc = a * d - b * c
  x = np.round((d * x_new  - b * y_new) / adbc).astype(np.int16) - tx + 1
  y = np.round((-c * x_new + a * y_new) / adbc).astype(np.int16) - ty + 1

  x = np.minimum(np.maximum(x, 0), W+1).astype(np.int16)
  y = np.minimum(np.maximum(y, 0), H+1).astype(np.int16)

  # assign value from original to affine image
  out[y_new, x_new] = _img[y, x]
  out = out.astype(np.uint8)

  return out

# Read image
img = cv2.imread("imori.jpg").astype(np.float32)

# Affine
out = affine(img, dx=30, dy=30)
cv2.imwrite("training_IMG/training_31.png", out)

## Q.32. フーリエ変換

二次元離散フーリエ変換(DFT)を実装し、*imori.jpg*をグレースケール化したものの周波数のパワースペクトルを表示せよ。
また、逆二次元離散フーリエ変換(IDFT)で画像を復元せよ。

二次元離散フーリエ変換(DFT: Discrete Fourier Transformation)とはフーリエ変換の画像に対する処理方法である。

通常のフーリエ変換はアナログ信号や音声などの連続値かつ一次元を対象に周波数成分を求める計算処理である。

一方、ディジタル画像は[0,255]の離散値をとり、かつ画像はHxWの二次元表示であるので、二次元離散フーリエ変換が行われる。

二次元離散フーリエ変換(DFT)は次式で計算される。

<!--
```bash
K = 0:W, l = 0:H, 入力画像をI として
G(k,l) = Sum_{y=0:H-1, x=0:W-1} I(x,y) exp( -2pi * j * (kx/W + ly/H)) / sqrt(H * W)
```
-->

K = [0, W-1], l = [0, H-1], 入力画像を I として

<img src="assets/dft_equ.png" width="500">

ここでは画像をグレースケール化してから二次元離散フーリエ変換を行え。

パワースペクトルとは Gは複素数で表されるので、Gの絶対値を求めることである。
今回のみ画像表示の時はパワースペクトルは[0,255]にスケーリングせよ。

逆二次元離散フーリエ変換(IDFT: Inverse DFT)とは周波数成分Gから元の画像を復元する手法であり、次式で定義される。

<!--
```bash
x = 0:W, y = 0:H  として
I(x,y) = Sum_{l=0:H-1, k=0:W-1} G(k,l) exp( 2pi * j * (kx/W + ly/H)) / sqrt(H * W)
```
-->
x = [0, W-1], y = [0, H-1] として

<img src="assets/idft_equ.png" width="500" >

上が定義式ですがexp(j)は複素数の値をとってしまうので、実際にコードにするときはぜ下式のように絶対値を使います。

<img src="assets/idct_equ2.png" width="500" >

シンプルに全部for文で回すと128^4の計算になるので、時間がかかってしまいます。numpyをうまく活用すれば計算コストを減らすことができます。（解答は128^2まで減らしました。）


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ハイパーパラメータの設定
K, L = 128, 128
channel = 3

def DFT(img):
    H, W, _ = img.shape
    G = np.zeros((L,K,channel), dtype=np.complex128)

    # オリジナルの画像に対応する2次元の座標を作成
    x = np.tile(np.arange(W),(H,1))
    y = np.arange(H).repeat(W).reshape(H,-1)

# ========================================================
# x = np.tile(np.arange(4), (3, 1))
# print(x)
# Output:
# [[0 1 2 3]
#  [0 1 2 3]
#  [0 1 2 3]]

# y = np.arange(3).repeat(4).reshape(3, -1)
# print(y)
# Output:
# [[0 0 0 0]
#  [1 1 1 1]
#  [2 2 2 2]]
# ========================================================

    # DFTの実装
    for c in range(channel):
        for l in range(L): # 画像の各行に対して処理を行う
            for k in range(K): # 画像の各列に対して処理を行う
                G[l,k,c] = np.sum(img[..., c]*np.exp(-2j*np.pi*(x*k / K+y*l / L))) / np.sqrt(K*L)
    return G

def IDFT(G):
    H, W, _ = G.shape
    out = np.zeros((H,W,channel), dtype=np.float32)

    # オリジナルの画像に対応する2次元の座標を作成
    x = np.tile(np.arange(W),(H,1))
    y = np.arange(H).repeat(W).reshape(H,-1)

    # IDFTの実装
    for c in range(channel):
        for l in range(H): # 画像の各行に対して処理を行う
            for k in range(W): # 画像の各列に対して処理を行う
                out[l,k,c] = np.abs(np.sum(G[..., c]*np.exp(2j*np.pi*(x*k / W+y*l / H)))) / np.sqrt(W*H)

    # クリッピング
    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)

    return out

# Read image
img = cv2.imread("imori.jpg").astype(np.float32)

# DFT
G = DFT(img)

# Save absolute value of DFT result as an image
G_abs = (np.abs(G) / np.abs(G).max() * 255).astype(np.uint8)
cv2.imwrite("training_IMG/training_32_dft.png", G_abs)

# IDFT
out = IDFT(G)
cv2.imwrite("training_IMG/training_32_idft.png", out)

## Q.33. フーリエ変換　ローパスフィルタ

*imori.jpg*をグレースケール化したものをDFTし、ローパスフィルタを通してIDFTで画像を復元せよ。

DFTによって得られた周波数成分は左上、右上、左下、右下に近いほど低周波数の成分を含んでいることになり、中心に近いほど高周波成分を示す。

![](assets/lpf.png)

画像における高周波成分とは色が変わっている部分（ノイズや輪郭など）を示し、低周波成分とは色があまり変わっていない部分（夕日のグラデーションなど）を表す。

ここでは、高周波成分をカットし、低周波成分のみを通す**ローパスフィルタ**を実装せよ。

ここでは低周波数の中心から高周波までの距離をrとすると0.5rまでの成分を通すとする。

※ ローパスフィルタ：信号処理において特定の頻度以下の信号は通過させ、それ以上の頻度の信号を遮断するフィルタのこと


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ハイパーパラメータの設定
K, L = 128, 128
channel = 3

def LPF(img, ratio=0.5):
  H, W, _ = img.shape

  # 位置の入れ替え、これによって低周波が中心に来るようにする。
  _img = np.zeros_like(img)
  _img[:H//2,:W//2] = img[H//2:,W//2:]
  _img[:H//2,W//2:] = img[H//2:,:W//2]
  _img[H//2:,:W//2] = img[:H//2,W//2:]
  _img[H//2:,W//2:] = img[:H//2,:W//2]

  # 中心からの距離を計算する
  x = np.tile(np.arange(W),(H,1))
  y = np.arange(H).repeat(W).reshape(H,-1)
  _x = x - W // 2
  _y = y - H // 2
  r = np.sqrt(_x**2 + _y**2)

  mask = np.ones((H,W), dtype=np.float32)
  mask[r>(W//2*ratio)] = 0
  mask = np.repeat(mask,channel).reshape(H,W,channel)

  # フィルタリング実行
  _img *= mask

  # 元の配置に戻す
  img[:H//2,:W//2] = _img[H//2:,W//2:]
  img[:H//2,W//2:] = _img[H//2:,:W//2]
  img[H//2:,:W//2] = _img[:H//2,W//2:]
  img[H//2:,W//2:] = _img[:H//2,:W//2]

  return img


# Read image
img = cv2.imread("imori.jpg").astype(np.float32)

# DFT
G = DFT(img)

# LPF
G = LPF(G)

# IDFT
out = IDFT(G)
out = np.clip(out, 0, 255).astype(np.uint8)

cv2.imwrite("training_IMG/training_33.png", out)

## Q.34. フーリエ変換　ハイパスフィルタ

*imori.jpg*をグレースケール化したものをDFTし、ハイパスフィルタを通してIDFTで画像を復元せよ。

ここでは、低周波成分をカットし、高周波成分のみを通す**ハイパスフィルタ**を実装せよ。

ここでは低周波数の中心から高周波までの距離をrとすると0.1rからの成分を通すとする。


In [ ]:
# ハイパーパラメータの設定
K, L = 128, 128
channel = 1

def HPF(img, ratio=0.5):
  H, W, _ = img.shape

  # 位置の入れ替え、これによって低周波が中心に来るようにする。
  _img = np.zeros_like(img)
  _img[:H//2,:W//2] = img[H//2:,W//2:]
  _img[:H//2,W//2:] = img[H//2:,:W//2]
  _img[H//2:,:W//2] = img[:H//2,W//2:]
  _img[H//2:,W//2:] = img[:H//2,:W//2]

  # 中心からの距離を計算する
  x = np.tile(np.arange(W),(H,1))
  y = np.arange(H).repeat(W).reshape(H,-1)
  _x = x - W // 2
  _y = y - H // 2
  r = np.sqrt(_x**2 + _y**2)

  mask = np.ones((H,W), dtype=np.float32)
  mask[r < (W//2*ratio)] = 0
  mask = np.repeat(mask,channel).reshape(H,W,channel)

  # フィルタリング実行
  _img *= mask

  # 元の配置に戻す
  img[:H//2,:W//2] = _img[H//2:,W//2:]
  img[:H//2,W//2:] = _img[H//2:,:W//2]
  img[H//2:,:W//2] = _img[:H//2,W//2:]
  img[H//2:,W//2:] = _img[:H//2,:W//2]

  return img

# Read image
img = cv2.imread("imori.jpg").astype(np.float32)

# グレースケール化
gray = 0.2126*img[...,2] + 0.7152*img[..., 1] + 0.0722*img[..., 0]
gray = np.expand_dims(gray, axis=-1)

# DFT
G = DFT(gray)

# HPF
G = HPF(G)

# IDFT
out = IDFT(G)

cv2.imwrite("training_IMG/training_34.png", out)

## Q.35. フーリエ変換　バンドパスフィルタ

*imori.jpg*をグレースケール化したものをDFTし、バンドパスフィルタを通してIDFTで画像を復元せよ。

ここでは、低周波成分と高周波成分の中間の周波数成分のみを通す**バンドパスフィルタ**を実装せよ。

ここでは低周波数の中心から高周波までの距離をrとすると0.1rから0.5rまでの成分を通すとする。


In [ ]:
# ハイパーパラメータの設定
K, L = 128, 128
channel = 1

def BPF(img,ratio_min=0.1,ratio_max=0.5):
  H, W, _ = img.shape

  # 位置の入れ替え、これによって低周波が中心に来るようにする。
  _img = np.zeros_like(img)
  _img[:H//2,:W//2] = img[H//2:,W//2:]
  _img[:H//2,W//2:] = img[H//2:,:W//2]
  _img[H//2:,:W//2] = img[:H//2,W//2:]
  _img[H//2:,W//2:] = img[:H//2,:W//2]

  # 中心からの距離を計算する
  x = np.tile(np.arange(W),(H,1))
  y = np.arange(H).repeat(W).reshape(H,-1)
  _x = x - W // 2
  _y = y - H // 2
  r = np.sqrt(_x**2 + _y**2)

  mask = np.ones((H,W), dtype=np.float32)
  mask[(r < (W//2*ratio_min)) | (r > (W//2*ratio_max))] = 0
  mask = np.repeat(mask,channel).reshape(H,W,channel)

  # フィルタリング実行
  _img *= mask

  # 元の配置に戻す
  img[:H//2,:W//2] = _img[H//2:,W//2:]
  img[:H//2,W//2:] = _img[H//2:,:W//2]
  img[H//2:,:W//2] = _img[:H//2,W//2:]
  img[H//2:,W//2:] = _img[:H//2,:W//2]

  return img

# Read image
img = cv2.imread("imori.jpg").astype(np.float32)

# グレースケール化
gray = 0.2126*img[...,2] + 0.7152*img[..., 1] + 0.0722*img[..., 0]
gray = np.expand_dims(gray, axis=-1)

# DFT
G = DFT(gray)

# BPF
G = BPF(G)

# IDFT
out = IDFT(G)

cv2.imwrite("training_IMG/training_35.png", out)

## Q.36. JPEG圧縮 (Step.1)離散コサイン変換

*imori.jpg*をグレースケール化し離散コサイン変換を行い、逆離散コサイン変換を行え。

離散コサイン変換(DCT: Discrete Cosine Transformation)とは、次式で定義される周波数変換の一つである。

<img src="assets/dct_equ.png" width="600" >

<!--
```bash
T = 8
F(u,v) = 2 / T * C(u)C(v) * Sum_{y=0:T-1} Sum_{x=0:T-1} f(x,y) cos((2x+1)u*pi/2T) cos((2y+1)v*pi/2T)
```
-->

逆離散コサイン変換(IDCT: Inverse Discrete Cosine Transformation)とは離散コサイン変換の逆（復号）であり、次式で定義される。

ここでいう K は復元時にどれだけ解像度を良くするかを決定するパラメータである。

K = Tの時は、DCT係数を全部使うのでIDCT後の解像度は最大になるが、Kが１や２などの時は復元に使う情報量（DCT係数）が減るので解像度が下がる。これを適度に設定することで、画像の容量を減らすことができる。

<img src="assets/idct_equ.png" width="600">

<!--
```bash
T = 8
K = 8
f(x,y) = 2 / T * C(x)C(y) * Sum_{u=0:K-1} Sum_{v=0:K-1} F(u,v) cos((2x+1)u*pi/2T) cos((2y+1)v*pi/2T)
```
-->

ここでは画像を8x8ずつの領域に分割して、各領域で以上のDCT, IDCTを繰り返すことで、jpeg符号に応用される。
今回も同様に8x8の領域に分割して、DCT, IDCTを行え。

In [ ]:
# ハイパーパラメータの設定
T, K = 8, 8
channel = 1

# 重みを計算する関数
def DCT_w(x,y,u,v):
  cu = 1.
  cv = 1.
  if u == 0:
    cu /= np.sqrt(2)
  if v == 0:
    cv /= np.sqrt(2)
  theta = np.pi/(2*T)
  return ((2*cu*cv/T)*np.cos((2*x+1)*u*theta)*np.cos((2*y+1)*v*theta))

def DCT(img):
  H, W, _ = img.shape
  F = np.zeros((H,W,channel), dtype=np.float32)

  # 画像を小さなブロックに分割した上で処理を施す
  for c in range(channel):
    for xi in range(0, H, T):
      for yi in range(0, W, T):
        for v in range(T):
          for u in range(T):
            for y in range(T):
              for x in range(T):
                F[v+yi,u+xi,c] += img[y+yi,x+xi,c]*DCT_w(x,y,u,v)
  return F

def IDCT(F):
  H, W, _ = F.shape
  out = np.zeros((H, W, channel), dtype=np.float32)

  for c in range(channel):
    for yi in range(0, H, T):
      for xi in range(0, W, T):
        for y in range(T):
          for x in range(T):
            for v in range(K):
              for u in range(K):
                out[y+yi,x+xi,c] += F[v+yi,u+xi,c]*DCT_w(x,y,u,v)

  out = np.clip(out, 0, 255)
  out = np.round(out).astype(np.uint8)

  return out

# Read image
img = cv2.imread("imori.jpg").astype(np.float32)

# グレースケール化
gray = 0.2126*img[...,2] + 0.7152*img[..., 1] + 0.0722*img[..., 0]
gray = np.expand_dims(gray, axis=-1)

# DCT
G = DCT(gray)

# IDCT
out = IDCT(G)

cv2.imwrite("training_IMG/training_36.png", out)